# Machine Learning Engineering (MLE)


[MLE](http://mlebook.com/) se refiere al diseño y construcción de software que incluye componentes basados en modelos de Machine Learning (ML). En este sentido, MLE se puede considerar como una extensión de la [ingeniería de software](https://es.wikipedia.org/wiki/Ingenier%C3%ADa_de_software) tradicional.

Hasta ahora nos hemos concentrado en presentar los aspectos más científicos de como entrenar y evaluar modelos de ML. MLE concierne no sólo el entrenamiento sino también aspectos relacionados a la colección y corrección de datos y al monitoreo y mantenimiendo de los modelos.

## Ciclo de vida de un proyecto de ML

Las tareas para desarrollar un sofware que utiliza ML pueden dividirse en:

1. Recolección y preparación de datos
1. Ingeniería de *features* (características)
1. Entrenamiento de modelos
1. Evaluación de modelos
1. Deployment de modelos
1. Servir modelos 
1. Monitoreo de modelos
1. Mantenimiento de modelos

Donde además se asume como "paso 0" la definición de uno o más objetivos, que vienen a ser los requisitos del software.

:::{important}

El objetivo debe definir las entradas y salidas del modelo. El objetivo también debe definir el criterio o métrica con la que se medirá que tan exitoso es el modelo.

:::

:::{note}

El movimiento entre los pasos anteriores puede ser "hacia atrás", por ejemplo ante una mala evaluación del modelo o errores detectados en el monitoreo, podríamos retroceder a recolectar nuevos datos y reentrenar el modelo.

:::

Los pasos 2, 3 y 4 han sido revisados en capítulos anteriores. A continuación revisaremos los demás pasos del flujo anterior. Pero antes respondamos la siguiente pregunta:


**¿Cuándo incorporar ML en mi software?**

ML provee herramientas para aprender modelos de predicción automáticamente a partir de datos. Antes de implementar corresponde preguntar si:

> ¿Es posible resolver el problema utilizando heurísticas o reglas programadas manualmente? 

En dicho caso no deberíamos utilizar ML en favor de software tradicional, por las complejidades adicionales que involucra mantener el primero. Otros condiciones que se deben cumplir para poder utilizar ML son:

- Tener una métrica objetivo bien definida
- Costos de obtener datos de ejemplo y etiquetarlos manejables
- Que sea admisible para el software entregar respuestas erróneas en algunas ocasiones (accuracy no 100%)

Otro indicio importante es si el problema en cuestion está relacionado a percepción humana, por ejemplo reconocimiento de patrones en imágenes (visión) o sonido (audición). En esos casos ML es típicamente la mejor solución.


## Recolección y preparación de datos



Antes de entrenar un modelo se necesitan datos. Antes de iniciar una campaña de recolección de datos que permitan cumplir el objetivo deberíamos investigar si:

- hay datos existentes que pueden utilizarse.
- los datos son suficientes para todas las clases/eventos de interés.
- los datos son relativamente actuales (no están obsoletos).
- los datos tienen calidad suficiente .

Los problemas típicamente encontrados en datasets de baja calidad son:

- Ruido en los datos.
- Ruido en las etiquetas: Etiquetadores inconsistentes.
- Datos faltantes (*missing data*).
- Sesgos de selección, muestreo, variables omitidas, etc.
- Presencia de *outliers*: Ejemplos muy alejados de la distribución que pueden afectar el entrenamiento.
- *data leakage*: La variable objetivo (etiqueta) está oculta en alguna de las variables.

Algunos de estos problemas puede resolverse mediante:

- Adecuado particionamiento de los datos.
- Inputación: Se refiere a rellenar datos faltantes utilizando reglas simples.
- Balanceo de clases mediante submuestreo (aleatorio o clustering), sobremuestreo (repetición) o aumentación sintética (por ejemplo [SMOTE](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)).
- Aumentación: Se refiere a crear sintéticamente nuevos datos basados en los existentes.


:::{seealso}

Para trabajar con datos desbalanceados utilizando los esquemas mencionados sugiero la librería [imbalanced-learn](https://imbalanced-learn.org/stable/index.html).

:::

Si no podemos resolver el problema usando estas técnicas sería necesario recolectar datos propios o re-etiquetar los datos existentes.

:::{seealso}

Para etiquetar datos de series de tiempo, texto, audio, imágenes de forma colaborativa y organizanda recomiendo el software [LabelStudio](https://labelstud.io/).

:::

Si los datos cambian con cierta frecuencia, viven en distintos servidores y/o necesitan ser compartidos y coordinados en un equipo conviene utilizar **versionamiento**.

El versionamiento de datos es un concepto reciente, muy similar al versionamiento de código:

- Cada cambio en nuestros datos es anotado mediante un *commit*.
- Los cambios hechos por distintas personas pueden coordinarse de forma centralizada.
- Se puede revisar fácilmente la historia de cambios y retroceder a un cambio anterior si es necesario.

:::{seealso}

Para realizar versionanamiento de datos sugiero la [Data Version Control](https://dvc.org/) (DVC), la cual está basada en *git*.

:::

## Herramientas de MLOps

Un concepto que se utiliza bastante hoy en día es [Machine Learning Operations](https://docs.microsoft.com/es-es/azure/machine-learning/concept-model-management-and-deployment) (MLOPs). MLOps se refiere a la implementación de prácticas de Development Operations (DevOps) en problemas de Machine Learning, entre ellas:

- Integración y Entrega continuas ([CI/CD](https://www.redhat.com/en/topics/devops/what-is-ci-cd))
- Flujos automáticos de trabajo (workflows)
- Control de versiones (códigos pero también datos y modelos)
- Empaquetado de soluciones
- Orquestación de carga de inferencia

:::{seealso}

Para hacer CI/CD con modelos de ML recomiendo [CML](https://cml.dev/).

:::

:::{seealso}

Para administrar experimentos de Machine Learning existen herramientas de linea de comando como [`dvc exp`](https://dvc.org/doc/command-reference/exp) o dashboards como [Neptune](https://neptune.ai/product), [MLFlow](https://mlflow.org/) o [Comet](https://www.comet.ml/site/).

:::

MLOps está relacionado a los últimos 4 pasos del esquema MLE, estos eran:

**Entrega y servicio de modelos** 

Se refiere a cómo se entrega o sirve el modelo y/o sus predicciones al usuario, por ejemplo:

- Estático: El modelo es parte de un ejecutable que está instalado en la máquina del usuario
- Dinámico (on-premise): El usuario tiene un cliente instalado en su máquina el cual solicita actualizaciones periódicas de los parámetros del modelo (cliente y modelo están separados)
- Dinámico (cloud): El usuario sube los datos a un servidor en la nube y recibe las predicciones

:::{note}

Los modelos de scikit-learn son amigables con [serialización](https://joblib.readthedocs.io/en/latest/generated/joblib.dump.html). El modelo serializado puede ser cargado por un servicio web basado en [Flask](https://flask.palletsprojects.com/en/2.1.x/) que luego el cliente accede a través de una API Rest. 

:::

:::{seealso}

La herramienta [MLEM](https://mlem.ai/) de Iterative.ai facilita considerablemente el flujo anterior.

:::

**Monitoreo y mantenimiento de modelos**

Se refiere a la inspección continua del desempeño del modelo en producción. Sea $X$ los datos de entrada al modelo e $Y$ la etiqueta a predecir, se define:

Data drift
: Situación en que la distribución $p(X)$ empieza a alejarse de la distribución original que se utilizó para entrenar el modelo. Esto puede deberse a que el problema es no-estacionario o a que hubo sesgo de muestreo al crear el dataset de entrenamiento

Prior probability shift
: Situación en que la distribución $p(Y)$ se aleja de la distribución original. Por ejemplo una clase deja de aparecer con tanta frecuencia y otra más originalmente más rara ocurre con mayor frecuencia. En el caso extremo podría aparecer una clase que originalmente no estaba en el dataset de entrenamiento.

Concept drift
: Situación en que la distribución condicional $p(Y|X)$ cambia con respecto a la original y por lo tanto nuestro mapeo $Y=f(X)$ pierde validez. Está relacionada a situaciones externas: cambio en la interpretación de la etiqueta, aparición de una clase que no estaba considerada originalmente, pérdida de poder predictivo.

:::{important}

Los *drift* puede identificarse comparando la distribución de los nuevos datos y predicciones con las de entrenamiento en base test estadísticos (por ejemplo Chi cuadrado o Kolmogorov-Smirov). En caso de detectar *drift*, debemos analizar las predicciones, etiquetar y reentrenar el modelo.

:::

Adicional al desempeño, también se debe monitorear:

- La disponibilidad del modelo
- La estabilidad numérica del modelo
- El consumo de recursos de hardware del modelo
